<!-- You can run this notebook in a Databricks environment. Specifically, this notebook has been designed to run in [Databricks Community Edition](http://community.cloud.databricks.com/) as well. -->
To run this notebook, you have to [create a cluster](https://docs.databricks.com/clusters/create.html) with version **Databricks Runtime 7.4 or later** and [attach this notebook](https://docs.databricks.com/notebooks/notebooks-manage.html#attach-a-notebook-to-a-cluster) to that cluster. <br/>

### Source Data for this notebook
The data used is a modified version of the public data from [Lending Club](https://www.kaggle.com/wendykan/lending-club-loan-data). It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).

## Setup

In [0]:
db = "deltadb"

spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")

spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

Out[7]: DataFrame[key: string, value: string]

In [0]:
import random
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *


def my_checkpoint_dir(): 
  return "/tmp/delta_demo/chkpt/%s" % str(random.randint(0, 10000))

# User-defined function to generate random state
@udf(returnType=StringType())
def random_state():
  return str(random.choice(["CA", "TX", "NY", "WA"]))


# Function to start a streaming query with a stream of randomly generated load data and append to the parquet table
def generate_and_append_data_stream(table_format, table_name, schema_ok=False, type="batch"):
  
  stream_data = (spark.readStream.format("rate").option("rowsPerSecond", 500).load()
    .withColumn("loan_id", 10000 + col("value"))
    .withColumn("funded_amnt", (rand() * 5000 + 5000).cast("integer"))
    .withColumn("paid_amnt", col("funded_amnt") - (rand() * 2000))
    .withColumn("addr_state", random_state())
    .withColumn("type", lit(type)))
    
  if schema_ok:
    stream_data = stream_data.select("loan_id", "funded_amnt", "paid_amnt", "addr_state", "type", "timestamp")
      
  query = (stream_data.writeStream
    .format(table_format)
    .option("checkpointLocation", my_checkpoint_dir())
    .trigger(processingTime = "5 seconds")
    .table(table_name))

  return query

In [0]:
# Function to stop all streaming queries 
def stop_all_streams():
    print("Stopping all streams")
    for s in spark.streams.active:
        try:
            s.stop()
        except:
            pass
    print("Stopped all streams")
    dbutils.fs.rm("/tmp/delta_demo/chkpt/", True)


def cleanup_paths_and_tables():
    dbutils.fs.rm("/tmp/delta_demo/", True)
    dbutils.fs.rm("file:/dbfs/tmp/delta_demo/loans_parquet/", True)
        
    for table in ["deltadb.loans_parquet", "deltadb.loans_delta", "deltadb.loans_delta2"]:
        spark.sql(f"DROP TABLE IF EXISTS {table}")
    
cleanup_paths_and_tables()

In [0]:
%sh mkdir -p /dbfs/tmp/delta_demo/loans_parquet/; wget -O /dbfs/tmp/delta_demo/loans_parquet/loans.parquet https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet

--2021-12-09 15:38:31-- https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet
Resolving pages.databricks.com (pages.databricks.com)... 104.17.71.206, 104.17.72.206, 104.17.73.206, ...
Connecting to pages.databricks.com (pages.databricks.com)|104.17.71.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164631 (161K) [text/plain]
Saving to: ‘/dbfs/tmp/delta_demo/loans_parquet/loans.parquet’

 0K .......... .......... .......... .......... .......... 31% 4.53M 0s
 50K .......... .......... .......... .......... .......... 62% 6.41M 0s
 100K .......... .......... .......... .......... .......... 93% 6.89M 0s
 150K .......... 100% 33.3M=0.03s

2021-12-09 15:38:31 (6.09 MB/s) - ‘/dbfs/tmp/delta_demo/loans_parquet/loans.parquet’ saved [164631/164631]

# Getting started with <img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>

An open-source storage layer for data lakes that brings ACID transactions to Apache Spark™ and big data workloads.

* **ACID Transactions**: Ensures data integrity and read consistency with complex, concurrent data pipelines.
* **Unified Batch and Streaming Source and Sink**: A table in Delta Lake is both a batch table, as well as a streaming source and sink. Streaming data ingest, batch historic backfill, and interactive queries all just work out of the box. 
* **Schema Enforcement and Evolution**: Ensures data cleanliness by blocking writes with unexpected.
* **Time Travel**: Query previous versions of the table by time or version number.
* **Deletes and upserts**: Supports deleting and upserting into tables with programmatic APIs.
* **Open Format**: Stored as Parquet format in blob storage.
* **Audit History**: History of all the operations that happened in the table.
* **Scalable Metadata management**: Able to handle millions of files are scaling the metadata operations with Spark.

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Convert to Delta Lake format

Delta Lake is 100% compatible with Apache Spark&trade;, which makes it easy to get started with if you already use Spark for your big data workflows.
Delta Lake features APIs for **SQL**, **Python**, and **Scala**, so that you can use it in whatever language you feel most comfortable in.

<img src="https://databricks.com/wp-content/uploads/2020/12/simplysaydelta.png" width=600/>

In **Python**: Read your data into a Spark DataFrame, then write it out in Delta Lake format directly, with no upfront schema definition needed.

In [0]:
parquet_path = "file:/dbfs/tmp/delta_demo/loans_parquet/"

df = (spark.read.format("parquet").load(parquet_path)
      .withColumn("type", lit("batch"))
      .withColumn("timestamp", current_timestamp()))

df.write.format("delta").mode("overwrite").saveAsTable("loans_delta")

**SQL:** Use `CREATE TABLE` statement with SQL (no upfront schema definition needed)

In [0]:
%sql
CREATE TABLE loans_delta2
USING delta
AS SELECT * FROM parquet.`/tmp/delta_demo/loans_parquet`

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Path does not exist: dbfs:/tmp/delta_demo/loans_parquet;; line 3 pos 17
	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:47)
	at org.apache.spark.sql.execution.datasources.ResolveSQLOnFile.org$apache$spark$sql$execution$datasources$ResolveSQLOnFile$$resolveDataSource(rules.scala:118)
	at org.apache.spark.sql.execution.datasources.ResolveSQLOnFile$$anonfun$apply$1.applyOrElse(rules.scala:142)
	at org.apache.spark.sql.execution.datasources.ResolveSQLOnFile$$anonfun$apply$1.applyOrElse(rules.scala:122)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$2(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:82)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:216)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:110)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$4(AnalysisHelper.scala:117)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$mapChildren$1(TreeNode.scala:414)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:252)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:412)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:365)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:117)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:216)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:110)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$4(AnalysisHelper.scala:117)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$mapChildren$1(TreeNode.scala:414)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:252)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:412)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:365)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:117)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:216)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:110)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperators(AnalysisHelper.scala:73)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperators$(AnalysisHelper.scala:72)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperators(LogicalPlan.scala:29)
	at org.apache.spark.sql.execution.datasources.ResolveSQLOnFile.apply(rules.scala:122)
	at org.apache.spark.sql.execution.datasources.ResolveSQLOnFile.apply(rules.scala:46)
	at org.ap

**SQL**: Use `CONVERT TO DELTA` to convert Parquet files to Delta Lake format in place

In [0]:
%sql CONVERT TO DELTA parquet.`/tmp/delta_demo/loans_parquet`

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: java.io.FileNotFoundException: No file found in the directory: dbfs:/tmp/delta_demo/loans_parquet.
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.emptyDirectoryException(DeltaErrors.scala:967)
	at com.databricks.sql.transaction.tahoe.commands.ConvertToDeltaCommandBase.$anonfun$performConvert$1(ConvertToDeltaCommand.scala:338)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:19)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:19)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:412)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:338)
	at com.databricks.spark.util.PublicDBLogging.recordOperation(DatabricksSparkUsageLogger.scala:19)
	at com.databricks.spark.util.PublicDBLogging.recordOperation0(DatabricksSparkUsageLogger.scala:56)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:129)
	at com.databricks.spark.util.UsageLogger.recordOperation(UsageLogger.scala:71)
	at com.databricks.spark.util.UsageLogger.recordOperation$(UsageLogger.scala:58)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:85)
	at com.databricks.spark.util.UsageLogging.recordOperation(UsageLogger.scala:401)
	at com.databricks.spark.util.UsageLogging.recordOperation$(UsageLogger.scala:380)
	at com.databricks.sql.transaction.tahoe.commands.ConvertToDeltaCommandBase.recordOperation(ConvertToDeltaCommand.scala:76)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordDeltaOperation(DeltaLogging.scala:108)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordDeltaOperation$(DeltaLogging.scala:94)
	at com.databricks.sql.transaction.tahoe.commands.ConvertToDeltaCommandBase.recordDeltaOperation(ConvertToDeltaCommand.scala:76)
	at com.databricks.sql.transaction.tahoe.commands.ConvertToDeltaCommandBase.performConvert(ConvertToDeltaCommand.scala:321)
	at com.databricks.sql.transaction.tahoe.commands.ConvertToDeltaCommandBase.$anonfun$run$1(ConvertToDeltaCommand.scala:113)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:1128)
	at com.databricks.sql.transaction.tahoe.commands.ConvertToDeltaCommandBase.run(ConvertToDeltaCommand.scala:85)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:79)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:234)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3709)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:116)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:249)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:101)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:845)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.ex

### View the data in the Delta Lake table
**How many records are there, and what does the data look like?**

In [0]:
spark.sql("select count(*) from loans_delta").show()
spark.sql("select * from loans_delta").show(3)

+--------+
count(1)|
+--------+
 14705|
+--------+

+-------+-----------+---------+----------+-----+--------------------+
loan_id|funded_amnt|paid_amnt|addr_state| type| timestamp|
+-------+-----------+---------+----------+-----+--------------------+
 0| 1000| 182.22| CA|batch|2021-12-09 15:38:...|
 1| 1000| 361.19| WA|batch|2021-12-09 15:38:...|
 2| 1000| 176.26| TX|batch|2021-12-09 15:38:...|
+-------+-----------+---------+----------+-----+--------------------+
only showing top 3 rows

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Unified batch + streaming data processing with multiple concurrent readers and writers

### Write 2 different data streams into our Delta Lake table at the same time.

In [0]:
# Set up 2 streaming writes to our table
stream_query_A = generate_and_append_data_stream(table_format="delta", table_name="loans_delta", schema_ok=True, type='stream A')
stream_query_B = generate_and_append_data_stream(table_format="delta", table_name="loans_delta", schema_ok=True, type='stream B')

### Create 2 continuous streaming readers of our Delta Lake table to illustrate streaming progress.

In [0]:
# Streaming read #1
display(spark.readStream.format("delta").table("loans_delta").groupBy("type").count().orderBy("type"))

type,count
batch,14705
stream A,167000
stream B,88500


In [0]:
# Streaming read #2
display(spark.readStream.format("delta").table("loans_delta").groupBy("type", window("timestamp", "10 seconds")).count().orderBy("window"))

type,window,count
batch,"List(2021-12-09T15:38:30.000+0000, 2021-12-09T15:38:40.000+0000)",14705
stream A,"List(2021-12-09T15:40:20.000+0000, 2021-12-09T15:40:30.000+0000)",3637
stream B,"List(2021-12-09T15:40:20.000+0000, 2021-12-09T15:40:30.000+0000)",2835
stream A,"List(2021-12-09T15:40:30.000+0000, 2021-12-09T15:40:40.000+0000)",5000
stream B,"List(2021-12-09T15:40:30.000+0000, 2021-12-09T15:40:40.000+0000)",5000
stream A,"List(2021-12-09T15:40:40.000+0000, 2021-12-09T15:40:50.000+0000)",5000
stream B,"List(2021-12-09T15:40:40.000+0000, 2021-12-09T15:40:50.000+0000)",5000
stream B,"List(2021-12-09T15:40:50.000+0000, 2021-12-09T15:41:00.000+0000)",5000
stream A,"List(2021-12-09T15:40:50.000+0000, 2021-12-09T15:41:00.000+0000)",5000
stream A,"List(2021-12-09T15:41:00.000+0000, 2021-12-09T15:41:10.000+0000)",5000


### Add a batch query, just for good measure

In [0]:
%sql
SELECT addr_state, COUNT(*)
FROM loans_delta
GROUP BY addr_state

addr_state,count(1)
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,178
VA,413
RI,66
WY,31


In [0]:
dbutils.notebook.exit("stop")

stop

In [0]:
stop_all_streams()

Stopping all streams
Stopped all streams

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) ACID Transactions

View the Delta Lake transaction log

In [0]:
%sql DESCRIBE HISTORY loans_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
41,2021-12-09T15:47:33.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 19)",null,List(400448375928372),1209-152421-tuxu0dh4,39,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 59500, numOutputBytes -> 1320218, numAddedFiles -> 1)",null
40,2021-12-09T15:47:24.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 20)",null,List(400448375928372),1209-152421-tuxu0dh4,39,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 20500, numOutputBytes -> 470357, numAddedFiles -> 1)",null
39,2021-12-09T15:46:39.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 18)",null,List(400448375928372),1209-152421-tuxu0dh4,37,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 58500, numOutputBytes -> 1298415, numAddedFiles -> 1)",null
38,2021-12-09T15:46:02.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 19)",null,List(400448375928372),1209-152421-tuxu0dh4,37,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 59500, numOutputBytes -> 1320176, numAddedFiles -> 1)",null
37,2021-12-09T15:45:20.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 18)",null,List(400448375928372),1209-152421-tuxu0dh4,35,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 24000, numOutputBytes -> 546702, numAddedFiles -> 1)",null
36,2021-12-09T15:44:03.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 17)",null,List(400448375928372),1209-152421-tuxu0dh4,35,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 8500, numOutputBytes -> 206293, numAddedFiles -> 1)",null
35,2021-12-09T15:43:23.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 17)",null,List(400448375928372),1209-152421-tuxu0dh4,33,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 5500, numOutputBytes -> 135037, numAddedFiles -> 1)",null
34,2021-12-09T15:43:12.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 16)",null,List(400448375928372),1209-152421-tuxu0dh4,33,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 4000, numOutputBytes -> 98935, numAddedFiles -> 1)",null
33,2021-12-09T15:43:06.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 16)",null,List(400448375928372),1209-152421-tuxu0dh4,31,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 5000, numOutputBytes -> 120139, numAddedFiles -> 1)",null
32,2021-12-09T15:43:01.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 15)",null,List(400448375928372),1209-152421-tuxu0dh4,31,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 6000, numOutputBytes -> 145211, numAddedFiles -> 1)",null


<img src="https://databricks.com/wp-content/uploads/2020/09/delta-lake-medallion-model-scaled.jpg" width=1012/>

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Use Schema Enforcement to protect data quality

To show you how schema enforcement works, let's create a new table that has an extra column -- `credit_score` -- that doesn't match our existing Delta Lake table schema.

#### Write DataFrame with extra column, `credit_score`, to Delta Lake table

In [0]:
# Generate `new_data` with additional column
new_column = [StructField("credit_score", IntegerType(), True)]
new_schema = StructType(spark.table("loans_delta").schema.fields + new_column)
data = [(99997, 10000, 1338.55, "CA", "batch", datetime.now(), 649),
        (99998, 20000, 1442.55, "NY", "batch", datetime.now(), 702)]

new_data = spark.createDataFrame(data, new_schema)
new_data.printSchema()

root
-- loan_id: long (nullable = true)
-- funded_amnt: integer (nullable = true)
-- paid_amnt: double (nullable = true)
-- addr_state: string (nullable = true)
-- type: string (nullable = true)
-- timestamp: timestamp (nullable = true)
-- credit_score: integer (nullable = true)

In [0]:
# Uncommenting this cell will lead to an error because the schemas don't match.
# Attempt to write data with new column to Delta Lake table
new_data.write.format("delta").mode("append").saveAsTable("loans_delta")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-400448375928409> in <module> 
 1 # Uncommenting this cell will lead to an error because the schemas don't match. 
 2 # Attempt to write data with new column to Delta Lake table 
 ----> 3 new_data . write . format ( "delta" ) . mode ( "append" ) . saveAsTable ( "loans_delta" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in saveAsTable (self, name, format, mode, partitionBy, **options) 
 866 if format is not None : 
 867 self . format ( format ) 
 --> 868 self . _jwrite . saveAsTable ( name ) 
 869 
 870 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : A schema mismatch detected when writing to the Delta table (Table ID: 652f730b-5d0d-49eb-b58f-9bcf5f7a123c).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- loan_id: long (nullable = true)
-- funded_amnt: integer (nullable = true)
-- paid_amnt: double (nullable = true)
-- addr_state: string (nullable = true)
-- type: string (nullable = true)
-- timestamp: timestamp (nullable = true)


Data schema:
root
-- loan_id: long (nullable = true)
-- funded_amnt: integer (nullable = true)
-- paid_amnt: double (nullable = true)
-- addr_state: string (nullable = true)
-- type: string (nullable = true)
-- timestamp: timestamp (nullable = true)
-- credit_score: integer (nullable = true)

 ;

**Schema enforcement helps keep our tables clean and tidy so that we can trust the data we have stored in Delta Lake.** The writes above were blocked because the schema of the new data did not match the schema of table (see the exception details). See more information about how it works [here](https://databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html).

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Use Schema Evolution to add new columns to schema

If we *want* to update our Delta Lake table to match this data source's schema, we can do so using schema evolution. Simply add the following to the Spark write command: `.option("mergeSchema", "true")`

In [0]:
new_data.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("loans_delta")

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id IN (99997, 99998)

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp,credit_score
99997,6301,5875.6904049278555,CA,stream B,2021-12-09T15:43:24.324+0000,null
99998,7893,6073.122435489927,TX,stream B,2021-12-09T15:43:24.326+0000,null
99998,5648,4025.023474160944,TX,stream A,2021-12-09T15:43:22.723+0000,null
99997,5580,4191.152712701205,CA,stream A,2021-12-09T15:43:22.721+0000,null
99997,10000,1338.55,CA,batch,2021-12-09T15:48:10.227+0000,649
99998,20000,1442.55,NY,batch,2021-12-09T15:48:10.227+0000,702


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake Time Travel

Delta Lake’s time travel capabilities simplify building data pipelines for use cases including:

* Auditing Data Changes
* Reproducing experiments & reports
* Rollbacks

As you write into a Delta table or directory, every operation is automatically versioned.

<img src="https://github.com/risan4841/img/blob/master/transactionallogs.png?raw=true" width=250/>

You can query snapshots of your tables by:
1. **Version number**, or
2. **Timestamp.**

using Python, Scala, and/or SQL syntax; for these examples we will use the SQL syntax.  

For more information, refer to the [docs](https://docs.delta.io/latest/delta-utility.html#history), or [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)

#### Review Delta Lake Table History for  Auditing & Governance
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts with schema modification

In [0]:
%sql
DESCRIBE HISTORY loans_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
42,2021-12-09T15:48:14.000+0000,492151203200240,kartiksmathur@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(400448375928372),1209-152421-tuxu0dh4,41,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 1969, numOutputRows -> 2)",null
41,2021-12-09T15:47:33.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 19)",null,List(400448375928372),1209-152421-tuxu0dh4,39,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 59500, numOutputBytes -> 1320218, numAddedFiles -> 1)",null
40,2021-12-09T15:47:24.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 20)",null,List(400448375928372),1209-152421-tuxu0dh4,39,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 20500, numOutputBytes -> 470357, numAddedFiles -> 1)",null
39,2021-12-09T15:46:39.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 18)",null,List(400448375928372),1209-152421-tuxu0dh4,37,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 58500, numOutputBytes -> 1298415, numAddedFiles -> 1)",null
38,2021-12-09T15:46:02.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 19)",null,List(400448375928372),1209-152421-tuxu0dh4,37,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 59500, numOutputBytes -> 1320176, numAddedFiles -> 1)",null
37,2021-12-09T15:45:20.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 18)",null,List(400448375928372),1209-152421-tuxu0dh4,35,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 24000, numOutputBytes -> 546702, numAddedFiles -> 1)",null
36,2021-12-09T15:44:03.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 17)",null,List(400448375928372),1209-152421-tuxu0dh4,35,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 8500, numOutputBytes -> 206293, numAddedFiles -> 1)",null
35,2021-12-09T15:43:23.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 17)",null,List(400448375928372),1209-152421-tuxu0dh4,33,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 5500, numOutputBytes -> 135037, numAddedFiles -> 1)",null
34,2021-12-09T15:43:12.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 10fb1838-3701-4e5a-bb06-aa90c20b0f7e, epochId -> 16)",null,List(400448375928372),1209-152421-tuxu0dh4,33,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 4000, numOutputBytes -> 98935, numAddedFiles -> 1)",null
33,2021-12-09T15:43:06.000+0000,492151203200240,kartiksmathur@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> bcc606b9-fc3c-4816-a6ed-ba62154b3be4, epochId -> 16)",null,List(400448375928372),1209-152421-tuxu0dh4,31,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 5000, numOutputBytes -> 120139, numAddedFiles -> 1)",null


#### Use time travel to select and view the original version of our table (Version 0).
As you can see, this version contains the original 14,705 records in it.

In [0]:
spark.sql("SELECT * FROM loans_delta VERSION AS OF 0").show(3)
spark.sql("SELECT COUNT(*) FROM loans_delta VERSION AS OF 0").show()

+-------+-----------+---------+----------+-----+--------------------+
loan_id|funded_amnt|paid_amnt|addr_state| type| timestamp|
+-------+-----------+---------+----------+-----+--------------------+
 0| 1000| 182.22| CA|batch|2021-12-09 15:38:...|
 1| 1000| 361.19| WA|batch|2021-12-09 15:38:...|
 2| 1000| 176.26| TX|batch|2021-12-09 15:38:...|
+-------+-----------+---------+----------+-----+--------------------+
only showing top 3 rows

+--------+
count(1)|
+--------+
 14705|
+--------+

In [0]:
%sql SELECT COUNT(*) FROM loans_delta

count(1)
408707


#### Rollback a table to a specific version using `RESTORE`

In [0]:
%sql RESTORE loans_delta VERSION AS OF 0

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
mismatched input 'RESTORE' expecting {'(', 'CONVERT', 'COPY', 'OPTIMIZE', 'ADD', 'ALTER', 'ANALYZE', 'CACHE', 'CLEAR', 'COMMENT', 'COMMIT', 'CREATE', 'DELETE', 'DESC', 'DESCRIBE', 'DFS', 'DROP', 'EXPLAIN', 'EXPORT', 'FROM', 'GRANT', 'IMPORT', 'INSERT', 'LIST', 'LOAD', 'LOCK', 'MAP', 'MERGE', 'MSCK', 'REDUCE', 'REFRESH', 'REPLACE', 'RESET', 'REVOKE', 'ROLLBACK', 'SELECT', 'SET', 'SHOW', 'START', 'TABLE', 'TRUNCATE', 'UNCACHE', 'UNLOCK', 'UPDATE', 'USE', 'VALUES', 'WITH'}(line 1, pos 0)

== SQL ==
RESTORE loans_delta VERSION AS OF 0
^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:266)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:133)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:55)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(ParseDriver.scala:81)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:70)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:90)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:67)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$2(SparkSession.scala:678)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:678)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:845)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:675)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:90)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:36)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:143)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:653)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:645)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:486)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:598)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:391)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:219)
	at java.lang.Thread.run(Thre

In [0]:
%sql SELECT COUNT(*) FROM loans_delta

count(1)
408707


##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support: `DELETE`, `UPDATE`, `MERGE INTO`

Delta Lake brings ACID transactions and full DML support to data lakes.

>Parquet does **not** support these commands - they are unique to Delta Lake.

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) `DELETE`: Handle GDPR or CCPA Requests on your Data Lake

Imagine that we are responding to a GDPR data deletion request. The user with loan ID #4420 wants us to delete their data. Here's how easy it is.

**View the user's data**

In [0]:
%sql
SELECT * FROM loans_delta WHERE loan_id=4420

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp,credit_score
4420,22000,1050.94,TX,batch,2021-12-09T15:38:32.015+0000,null


**Delete the individual user's data with a single `DELETE` command using Delta Lake.**

Note: The `DELETE` command isn't supported in Parquet.

In [0]:
%sql
DELETE FROM loans_delta WHERE loan_id=4420;
-- Confirm the user's data was deleted
SELECT * FROM loans_delta WHERE loan_id=4420

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp,credit_score


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Use time travel and `INSERT INTO` to add the user back into our table

In [0]:
%sql
INSERT INTO loans_delta
SELECT * FROM loans_delta VERSION AS OF 0
WHERE loan_id=4420

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Cannot write to 'deltadb.loans_delta', not enough data columns; target table has 7 column(s) but the inserted data has 6 column(s);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notEnoughColumnsInInsert(DeltaErrors.scala:372)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis.com$databricks$sql$transaction$tahoe$DeltaAnalysis$$needsSchemaAdjustment(DeltaAnalysis.scala:397)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:60)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:57)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$2(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:82)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:216)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:110)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis.apply(DeltaAnalysis.scala:57)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis.apply(DeltaAnalysis.scala:52)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:152)
	at scala.collection.LinearSeqOptimized.foldLeft(LinearSeqOptimized.scala:126)
	at scala.collection.LinearSeqOptimized.foldLeft$(LinearSeqOptimized.scala:122)
	at scala.collection.immutable.List.foldLeft(List.scala:89)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:149)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1$adapted(RuleExecutor.scala:141)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:141)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:217)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:180)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$executeAndTrack$1(RuleExecutor.scala:119)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:119)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:201)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:200)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:845)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:171)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:92)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:89)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:81)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id=4420

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp,credit_score


### ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) `UPDATE`: Modify the existing records in a table in one command

In [0]:
%sql UPDATE loans_delta SET funded_amnt = 22000 WHERE loan_id = 4420

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id = 4420

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp,credit_score


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Support Change Data Capture Workflows & Other Ingest Use Cases via `MERGE INTO`

With a legacy data pipeline, to insert or update a table, you must:
1. Identify the new rows to be inserted
2. Identify the rows that will be replaced (i.e. updated)
3. Identify all of the rows that are not impacted by the insert or update
4. Create a new temp based on all three insert statements
5. Delete the original table (and all of those associated files)
6. "Rename" the temp table back to the original table name
7. Drop the temp table

<img src="https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif" alt='Merge process' width=600/>


#### INSERT or UPDATE with Delta Lake

2-step process: 
1. Identify rows to insert or update
2. Use `MERGE`

In [0]:
# Create merge table with 1 row update, 1 insertion
data = [(4420, 22000, 21500.00, "NY", "update", datetime.now()),  # record to update
        (99999, 10000, 1338.55, "CA", "insert", datetime.now())]  # record to insert
schema = spark.table("loans_delta").schema
spark.createDataFrame(data, schema).createOrReplaceTempView("merge_table")
spark.sql("SELECT * FROM merge_table").show()

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-400448375928439> in <module> 
 3 (99999, 10000, 1338.55, "CA", "insert", datetime.now())] # record to insert
 4 schema = spark . table ( "loans_delta" ) . schema
 ----> 5 spark . createDataFrame ( data , schema ) . createOrReplaceTempView ( "merge_table" ) 
 6 spark . sql ( "SELECT * FROM merge_table" ) . show ( ) 

 /databricks/spark/python/pyspark/databricks/utils/instrumentation.py in wrapper (self, *args, **kwargs) 
 40 try : 
 41 start_time = time . time ( ) 
 ---> 42 return_val = func ( self , * args , ** kwargs ) 
 43 except Exception : 
 44 duration = ( time . time ( ) - start_time ) * 1000 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 656 return super(SparkSession, self).createDataFrame(
 657 data, schema, samplingRatio, verifySchema)
 --> 658 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 
 659 
 660 def _create_dataframe ( self , data , schema , samplingRatio , verifySchema ) : 

 /databricks/spark/python/pyspark/sql/session.py in _create_dataframe (self, data, schema, samplingRatio, verifySchema) 
 688 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 689 else : 
 --> 690 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 691 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 692 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 472 write temp files . 
 473 """
 --> 474 data , schema = self . _wrap_data_schema ( data , schema ) 
 475 return self . _sc . parallelize ( data ) , schema
 476 

 /databricks/spark/python/pyspark/sql/session.py in _wrap_data_schema (self, data, schema) 
 448 # make sure data could consumed multiple times 
 449 if not isinstance ( data , list ) : 
 --> 450 data = list ( data ) 
 451 
 452 if schema is None or isinstance ( schema , ( list , tuple ) ) : 

 /databricks/spark/python/pyspark/sql/session.py in prepare (obj) 
 663 
 664 def prepare ( obj ) : 
 --> 665 verify_func ( obj ) 
 666 return obj
 667 elif isinstance ( schema , DataType ) : 

 /databricks/spark/python/pyspark/sql/types.py in verify (obj) 
 1406 def verify ( obj ) : 
 1407 if not verify_nullability ( obj ) : 
 -> 1408 verify_value ( obj ) 
 1409 
 1410 return verify

 /databricks/spark/python/pyspark/sql/types.py in verify_struct (obj) 
 1385 raise ValueError(
 1386 new_msg("Length of object (%d) does not match with "
 -> 1387 "length of fields (%d)" % (len(obj), len(verifiers))))
 1388 for v , ( _ , verifier ) in zip ( obj , verifiers ) : 
 1389 verifier ( v ) 

 ValueError : Length of object (6) does not match with length of fields (7)

In [0]:
%sql
MERGE INTO loans_delta AS l
USING merge_table AS m
ON l.loan_id = m.loan_id
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *;

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: merge_table; line 1 pos 0;
'MergeIntoTable ('l.loan_id = 'm.loan_id), [updateaction(None)], [insertaction(None)]
:- SubqueryAlias l
: +- SubqueryAlias spark_catalog.deltadb.loans_delta
: +- Relation[loan_id#16205L,funded_amnt#16206,paid_amnt#16207,addr_state#16208,type#16209,timestamp#16210,credit_score#16211] parquet
+- 'SubqueryAlias m
 +- 'UnresolvedRelation [merge_table]

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:109)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1$adapted(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:192)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:191)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:191)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:191)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:191)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:191)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:191)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:92)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:180)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:203)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:200)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:845)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:171)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:92)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:89)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:81)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:845)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:680)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:845)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:675)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:90)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:36)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:143)
	at com.databricks.backend.daemon.d

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id IN (4420, 99999)

loan_id,funded_amnt,paid_amnt,addr_state,type,timestamp,credit_score
99999,5755,3905.461749111245,WA,stream B,2021-12-09T15:43:24.328+0000,null
99999,6768,5637.678147915814,TX,stream A,2021-12-09T15:43:22.725+0000,null


## ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) File compaction and performance optimizations = faster queries

### Vacuum

In [0]:
%sql
-- Vacuum deletes all files no longer needed by the current version of the table.
VACUUM loans_delta

path
dbfs:/user/hive/warehouse/deltadb.db/loans_delta


### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Cache table in memory (Databricks Delta Lake only)

In [0]:
%sql CACHE SELECT * FROM loans_delta

### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Z-Order Optimize (Databricks Delta Lake only)

In [0]:
%sql OPTIMIZE loans_delta ZORDER BY addr_state

path,metrics
null,"List(1, 43, List(8264259, 8264259, 8264259.0, 1, 8264259), List(789, 1320218, 217049.0, 43, 9333145), 0, List(minCubeSize(107374182400), List(0, 0), List(43, 9333145), 0, List(43, 9333145), 1, null), 1)"


In [0]:
cleanup_paths_and_tables()